In [1]:
%load_ext pycodestyle_magic
%flake8_on --max_line_length 120

In [2]:
import findspark
findspark.init()

In [16]:
import pickle
import numpy as np

In [6]:
from source.als_recommender import AlternatingLeastSquares
from source import utils

In [7]:
spark = utils.create_spark_session()
train, test = utils.prepare_data(spark, 100000)

In [8]:
k = 5
als = AlternatingLeastSquares()

In [11]:
# als.grid_search(train, rank=[10, 20], max_iter=[10],
#                 reg_param=np.linspace(0.05, 0.3, 10), num_folds=2, metric='rmse')

als.grid_search(train, rank=[2], max_iter=[2],
                reg_param=np.linspace(0.05, 0.3, 2), num_folds=2,
                metric='rmse', parallelism=2)

4:47: W291 trailing whitespace
5:46: W291 trailing whitespace


In [12]:
pred_ratings, pred_rankings = als.predict(test)

pd_pred_ratings = pred_ratings.toPandas()
pd_pred_rankings = pred_rankings.toPandas()

rmse_by_user = utils.rmse_distribution(pd_pred_ratings, 'userId')
rmse_by_movie = utils.rmse_distribution(pd_pred_ratings, 'movieId')

top_k_precision_by_user = utils.top_k_precision_distribution(pd_pred_rankings, k)

with open('data/results/als_distribution_rmse_user.pkl', 'wb') as f:
    pickle.dump([rmse_by_user, 'als_distribution_rmse_user'], f)

with open('data/results/als_distribution_rmse_movie.pkl', 'wb') as f:
    pickle.dump([rmse_by_movie, 'als_distribution_rmse_movie'], f)

with open('data/results/als_distribution_topk_user.pkl', 'wb') as f:
    pickle.dump([top_k_precision_by_user, 'als_distribution_topk_user'], f)

In [26]:
experiment_title = 'als_hyperparameter_tuning_for_reg_param_rmse'
reg_params = [r['regParam'] for r in als.results]
rmses = [r['rmse'] for r in als.results]

result = [experiment_title, pd.Series(reg_params), pd.Series(rmses)]
with open('data/results/als_hyperparameter_tuning_for_reg_param_rmse.pkl', 'wb') as f:
    pickle.dump(result, f)

In [ ]:
als.rmse()

In [ ]:
als.precision_at_k(k)